In [1]:
# ELO Merchant Category Recommendation - Exploratory Data Analysis (Merchant.csv)

In [2]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from IPython.display import display

# Set the visualization style
sns.set(style="whitegrid")
plt.style.use('fivethirtyeight')


# Define data path - adjust as needed
DATA_PATH = '../data/raw/'  # Add your path here if needed

In [3]:
# Cell 2: Load Merchant Data
print("\n=== Loading Merchant Data ===")
merchant = pd.read_csv(DATA_PATH + 'merchants.csv')
print(f"Merchant dataset shape: {merchant.shape}")


=== Loading Merchant Data ===
Merchant dataset shape: (334696, 22)


In [4]:
# Cell 3: Basic Merchant Data Overview
print("\n=== Merchant Data Overview ===")
print(merchant.head())
print("\nColumn information:")
print(merchant.info())



=== Merchant Data Overview ===
       merchant_id  merchant_group_id  merchant_category_id  subsector_id  \
0  M_ID_838061e48c               8353                   792             9   
1  M_ID_9339d880ad               3184                   840            20   
2  M_ID_e726bbae1e                447                   690             1   
3  M_ID_a70e9c5f81               5026                   792             9   
4  M_ID_64456c37ce               2228                   222            21   

   numerical_1  numerical_2 category_1 most_recent_sales_range  \
0    -0.057471    -0.057471          N                       E   
1    -0.057471    -0.057471          N                       E   
2    -0.057471    -0.057471          N                       E   
3    -0.057471    -0.057471          Y                       E   
4    -0.057471    -0.057471          Y                       E   

  most_recent_purchases_range  avg_sales_lag3  ...  avg_sales_lag6  \
0                           E         

In [6]:
# Cell 4: Check Missing Values
print("\n=== Missing Values Analysis ===")
missing_values = merchant.isnull().sum().sort_values(ascending=False)
missing_percent = (merchant.isnull().sum() / merchant.shape[0] * 100).sort_values(ascending=False)
missing_df = pd.concat([missing_values, missing_percent], axis=1, keys=['Missing Values', 'Missing Percentage'])
print(missing_df[missing_df['Missing Values'] > 0])


=== Missing Values Analysis ===
                 Missing Values  Missing Percentage
category_2                11887            3.551581
avg_sales_lag3               13            0.003884
avg_sales_lag6               13            0.003884
avg_sales_lag12              13            0.003884


In [7]:
# Cell 5: Categorical Columns Analysis
print("\n=== Categorical Columns Analysis ===")
categorical_cols = merchant.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(merchant[col].value_counts(dropna=False))
    print(f"Unique values: {merchant[col].nunique()}")


=== Categorical Columns Analysis ===

Column: merchant_id
merchant_id
M_ID_dbbf07ebf0    4
M_ID_6464db3b45    4
M_ID_30340088f2    4
M_ID_d123532c72    4
M_ID_ebbdb42da6    4
                  ..
M_ID_3a5c276d84    1
M_ID_93490b5f91    1
M_ID_0997eb7ae1    1
M_ID_d41360c62c    1
M_ID_dd326c2f05    1
Name: count, Length: 334633, dtype: int64
Unique values: 334633

Column: category_1
category_1
N    327657
Y      7039
Name: count, dtype: int64
Unique values: 2

Column: most_recent_sales_range
most_recent_sales_range
E    177104
D    117475
C     34075
B      5037
A      1005
Name: count, dtype: int64
Unique values: 5

Column: most_recent_purchases_range
most_recent_purchases_range
E    175309
D    119187
C     34144
B      5046
A      1010
Name: count, dtype: int64
Unique values: 5

Column: category_4
category_4
N    238596
Y     96100
Name: count, dtype: int64
Unique values: 2


In [9]:
# Cell 12: Check for Duplicate Merchant IDs
print("\n=== Checking Duplicate Merchant IDs ===")

# Count occurrences of each merchant_id
id_counts = merchant['merchant_id'].value_counts()

# Print merchant_ids that appear more than once
duplicate_ids = id_counts[id_counts > 1]
print(f"Found {len(duplicate_ids)} merchant IDs with duplicates")
print(f"Total duplicate rows: {sum(duplicate_ids) - len(duplicate_ids)}")
print("\nTop duplicate merchant IDs:")
print(duplicate_ids.head(10))

# Get the full records of these duplicate merchant IDs for inspection
duplicate_merchants = merchant[merchant['merchant_id'].isin(duplicate_ids.index)]
duplicate_merchants = duplicate_merchants.sort_values('merchant_id')

# Check if the duplicate entries are completely identical or have differences
print("\nExample of duplicate entries for one merchant ID:")
example_id = duplicate_ids.index[0]
print(merchant[merchant['merchant_id'] == example_id])

# Check if duplicate entries differ in any columns
print("\nChecking for differences in duplicate entries:")
for merchant_id in duplicate_ids.index[:5]:  # Check first 5 duplicate IDs
    df_subset = merchant[merchant['merchant_id'] == merchant_id].reset_index(drop=True)
    print(f"\nMerchant ID: {merchant_id}, {len(df_subset)} occurrences")
    
    # Check if all rows are identical
    if df_subset.equals(pd.DataFrame([df_subset.iloc[0].values] * len(df_subset), columns=df_subset.columns)):
        print("  All entries are identical")
    else:
        # Find columns where values differ
        differing_cols = []
        for col in df_subset.columns:
            if not df_subset[col].equals(df_subset[col].iloc[0]):
                differing_cols.append(col)
        print(f"  Entries differ in columns: {differing_cols}")
        
        # Show the differing values
        for col in differing_cols:
            print(f"  Values in '{col}':")
            for i, value in enumerate(df_subset[col]):
                print(f"    Row {i}: {value}")


=== Checking Duplicate Merchant IDs ===
Found 41 merchant IDs with duplicates
Total duplicate rows: 63

Top duplicate merchant IDs:
merchant_id
M_ID_dbbf07ebf0    4
M_ID_6464db3b45    4
M_ID_30340088f2    4
M_ID_d123532c72    4
M_ID_ebbdb42da6    4
M_ID_42697d5d44    4
M_ID_1802942aaf    4
M_ID_c2b9ac2ea4    4
M_ID_ef233cff26    4
M_ID_992a180b15    4
Name: count, dtype: int64

Example of duplicate entries for one merchant ID:
          merchant_id  merchant_group_id  merchant_category_id  subsector_id  \
7584  M_ID_dbbf07ebf0              69462                   278            37   
7585  M_ID_dbbf07ebf0                 35                   278            37   
7586  M_ID_dbbf07ebf0              69462                   278            37   
7587  M_ID_dbbf07ebf0                 35                   278            37   

      numerical_1  numerical_2 category_1 most_recent_sales_range  \
7584    -0.057471    -0.057471          N                       E   
7585    -0.057471    -0.05747

In [10]:
# Cell 14: Analyze duplicate patterns more deeply
print("\n=== Further Duplicate Analysis ===")

# Check how many duplicate merchants have 35 as one of their merchant_group_ids
duplicate_with_grp_35 = 0
for m_id in duplicate_ids.index:
    if 35 in merchant[merchant['merchant_id'] == m_id]['merchant_group_id'].values:
        duplicate_with_grp_35 += 1

print(f"Duplicate merchants with group_id 35: {duplicate_with_grp_35} out of {len(duplicate_ids)}")

# Check if duplicates generally come in pairs with the same number of rows
duplicate_counts = id_counts[id_counts > 1].value_counts()
print("\nCounts of duplicate occurrences:")
print(duplicate_counts)

# Cell 15: Handle Duplicates
print("\n=== Handling Duplicates ===")

# Strategy 1: Remove entries with merchant_group_id = 35 if duplicated
merchant_clean = merchant.copy()
dupe_mask = merchant['merchant_id'].isin(duplicate_ids.index)
grp_35_mask = merchant['merchant_group_id'] == 35
remove_mask = dupe_mask & grp_35_mask

print(f"Removing {remove_mask.sum()} rows with duplicated merchant_id and group_id 35")
merchant_clean = merchant_clean[~remove_mask]
print(f"Shape after removing group 35 duplicates: {merchant_clean.shape}")

# Check if we've resolved all duplicates
remaining_dupes = merchant_clean['merchant_id'].value_counts()
remaining_dupes = remaining_dupes[remaining_dupes > 1]
print(f"Remaining duplicated merchant IDs: {len(remaining_dupes)}")

if len(remaining_dupes) > 0:
    # Strategy 2: For any remaining duplicates, keep the first occurrence
    print("Removing remaining duplicates by keeping first occurrence")
    merchant_clean = merchant_clean.drop_duplicates(subset=['merchant_id'], keep='first')
    print(f"Final shape after removing all duplicates: {merchant_clean.shape}")

# Verify no duplicates remain
final_check = merchant_clean['merchant_id'].value_counts()
final_dupes = final_check[final_check > 1]
print(f"Final duplicate check: {len(final_dupes)} duplicated merchant IDs")


=== Further Duplicate Analysis ===
Duplicate merchants with group_id 35: 40 out of 41

Counts of duplicate occurrences:
count
2    30
4    11
Name: count, dtype: int64

=== Handling Duplicates ===
Removing 80 rows with duplicated merchant_id and group_id 35
Shape after removing group 35 duplicates: (334616, 22)
Remaining duplicated merchant IDs: 12
Removing remaining duplicates by keeping first occurrence
Final shape after removing all duplicates: (334604, 22)
Final duplicate check: 0 duplicated merchant IDs


In [8]:
# Cell 6: Numerical Columns Distribution
print("\n=== Numerical Columns Distribution ===")
numerical_cols = merchant.select_dtypes(include=['float64', 'int64']).columns
merchant[numerical_cols].describe().T


=== Numerical Columns Distribution ===


/home/psl/anaconda3/envs/SC4000/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/psl/anaconda3/envs/SC4000/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/psl/anaconda3/envs/SC4000/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,count,mean,std,min,25%,50%,75%,max
merchant_group_id,334696.0,3.102874e+04,31623.043426,1.000000,3612.000000,19900.000000,51707.250000,1.125860e+05
merchant_category_id,334696.0,4.231317e+02,252.898046,-1.000000,222.000000,373.000000,683.000000,8.910000e+02
subsector_id,334696.0,2.511640e+01,9.807371,-1.000000,19.000000,27.000000,33.000000,4.100000e+01
numerical_1,334696.0,1.147638e-02,1.098154,-0.057471,-0.057471,-0.057471,-0.047556,1.837351e+02
numerical_2,334696.0,8.103110e-03,1.070497,-0.057471,-0.057471,-0.057471,-0.047556,1.820793e+02
avg_sales_lag3,334683.0,1.383299e+01,2395.489999,-82.130000,0.880000,1.000000,1.160000,8.518446e+05
avg_purchases_lag3,334696.0,inf,NaN,0.333495,0.923650,1.016667,1.146522,inf
active_months_lag3,334696.0,2.994108e+00,0.095247,1.000000,3.000000,3.000000,3.000000,3.000000e+00
avg_sales_lag6,334683.0,2.165079e+01,3947.108099,-82.130000,0.850000,1.010000,1.230000,1.513959e+06
avg_purchases_lag6,334696.0,inf,NaN,0.167045,0.902247,1.026961,1.215575,inf
